In [1]:
from ase import Atoms
from ase.build import bulk, make_supercell
import random
import math
import numpy as np
import os
from chgnet.model.model import CHGNet 
from chgnet.model.dynamics import CHGNetCalculator 
from chgnet.model import StructOptimizer
import json

# Functions

In [2]:
def generate_random_supercells(composition, num_structures, lattice_parameter=3.01, supercell_size=2):
    supercells = []

    supercell_matrix=[[supercell_size, 0, 0], [0, supercell_size, 0], [0, 0, supercell_size]]
    comp_list = [key for key in composition]
    for _ in range(num_structures):
        # Create a bulk V crystal with the specified lattice parameter
        #prim_cell = bulk(comp_list[0], cubic=True, a=lattice_parameter)
        prim_cell = bulk(comp_list[0],a=lattice_parameter)

        # Get the unit cell and multiply it to create the supercell
        supercell = make_supercell(prim_cell , supercell_matrix)

        for element, count in composition.items():
            # Create a list of all possible indices for the current element
            all_indices = list(range(len(supercell)))
            # Randomly select 'count' indices for the current element
            selected_indices = random.sample(all_indices, count)
            for index in selected_indices:
                supercell[index].symbol = element

        supercells.append(supercell)

    return supercells

def create_vacancy(structure: Atoms, index: int) -> Atoms:
    new_structure = structure.copy()
    del new_structure[index]
    return new_structure

def replace_atom(structure: Atoms, index: int, new_atom: str) -> Atoms:
    new_structure = structure.copy()
    new_structure[index].symbol = new_atom
    return new_structure

def create_vacancies_in_direction(structure: Atoms, index: int, direction: np.array) -> (Atoms, Atoms):
    # Create a copy of the structure to avoid modifying the original
    structure_copy = structure.copy()

    # Create the first vacancy
    #vacancy1 = create_vacancy(structure_copy, index)
    vacancy1 = replace_atom(structure_copy, index, 'O')

    # Find the distances and vectors to all other atoms
    indices = [i for i in range(len(structure)) if i != index]
    #print(index)
    #print("indices here")
    #print(indices)
    vectors = structure.get_distances(index, indices, mic=True, vector=True)

    # Project the vectors onto the specified direction
    projections = np.dot(vectors, direction)

    # Find the index of the atom with the largest projection in the specified direction
    index2 = indices[np.argmax(projections)]

    # Create a copy of the original structure again for the second vacancy
    structure_copy2 = structure.copy()

    # Create the second vacancy
    #vacancy2 = create_vacancy(structure_copy2, index2)
    vacancy2 = replace_atom(structure_copy2, index2, 'O')

    return vacancy1, vacancy2


# Run Code

In [3]:
# create the supercells:

x_cr = 0.02
x_ti  = 0.03
x_w = 0.05
x_zr  = 0.02
x_ta = 0.01
x_v = 1 - x_cr - x_ti - x_w - x_zr - x_ta
supercell_size = 5
num_atoms = supercell_size**3 
num_cr  = math.ceil(num_atoms * x_cr)
num_ti = math.ceil(num_atoms * x_ti)
num_w  = math.ceil(num_atoms * x_w)
num_zr = math.ceil(num_atoms * x_zr)
num_ta = math.ceil(num_atoms * x_ta)
num_v = supercell_size**3 - num_cr - num_ti - num_zr - num_w - num_ta
atom_dict = {'V' : num_v , 'Cr' : num_cr, 'Ti' : num_ti , 'Zr' : num_zr, 'W' : num_w, 'Ta' : num_ta}
num_structures = 10 
generated_supercells = generate_random_supercells(atom_dict, num_structures, supercell_size=supercell_size)

# Access the generated supercells in the list
#for i, supercell in enumerate(generated_supercells):
    #supercell.write(f'supercell_{i}.xyz')

In [68]:
# select model 
chgnet_vac_jan_26_pot = CHGNet.from_file('/Users/myless/Packages/auto-n3b/Vacancy_Train_Results/bestF_epoch89_e2_f28_s55_mNA.pth.tar')

CHGNet v0.3.0 initialized with 412,525 parameters


In [69]:
# Create a new list to store the relaxed structures
relaxed_structures = []

# Relax the supercells and add the relaxed structures to the new list
for i, supercell in enumerate(generated_supercells):
    # Create a copy of the supercell
    pre_relaxed_supercell = supercell.copy()
    
    # Set the calculator to the CHGNetCalculator
    #relaxed_supercell.calc = CHGNetCalculator(chgnet_vac_jan_26_pot)
    relaxer = StructOptimizer(chgnet_vac_jan_26_pot)
    relaxed_supercell = relaxer.relax(atoms = pre_relaxed_supercell, fmax = 0.01, relax_cell=True)
    
    # Relax the structure
    relaxed_structure = relaxed_supercell['final_structure']
    relax_energy = relaxed_supercell['trajectory'].energies[-1]
    relax_force = relaxed_supercell['trajectory'].forces[-1]
    relax_stress = relaxed_supercell['trajectory'].stresses[-1]
    #relaxed_supercell.write('relaxed_{}.cif'.format(i))
    print(relaxed_structure)
    results = {'structure' : relaxed_structure, 'energy' : relax_energy, 'force' : relax_force, 'stress' : relax_stress} 
    # Add the relaxed structure to the new list
    relaxed_structures.append(results)


CHGNet will run on cpu
      Step     Time          Energy          fmax
FIRE:    0 15:34:53    -1142.615557        1.619931
FIRE:    1 15:34:53    -1142.986178        1.267673
FIRE:    2 15:34:54    -1143.579483        0.952079
FIRE:    3 15:34:54    -1144.193411        0.713655
FIRE:    4 15:34:55    -1144.667268        0.497689
FIRE:    5 15:34:55    -1144.929647        0.481273
FIRE:    6 15:34:56    -1145.019293        0.472496
FIRE:    7 15:34:56    -1145.025373        0.471777
FIRE:    8 15:34:57    -1145.037293        0.470409
FIRE:    9 15:34:58    -1145.054221        0.468481
FIRE:   10 15:34:58    -1145.075560        0.465921
FIRE:   11 15:34:59    -1145.099878        0.462705
FIRE:   12 15:34:59    -1145.126104        0.459254
FIRE:   13 15:35:00    -1145.153284        0.455384
FIRE:   14 15:35:00    -1145.183206        0.450736
FIRE:   15 15:35:01    -1145.214438        0.444733
FIRE:   16 15:35:02    -1145.246863        0.437252
FIRE:   17 15:35:02    -1145.280719        

In [54]:
test_structure = relaxed_structures[0]['structure']
print(test_structure)

from pymatgen.io.ase import AseAtomsAdaptor

# Assuming `structure` is your pymatgen Structure object
adaptor = AseAtomsAdaptor()
ase_atoms = adaptor.get_atoms(test_structure)

Full Formula (Zr1 Ta1 Ti4 V110 Cr2 W7)
Reduced Formula: ZrTaTi4V110Cr2W7
abc   :  13.100481  13.108716  13.100538
angles: 109.486982 109.450323 109.502680
pbc   :       True       True       True
Sites (125)
  #  SP            a          b          c       magmom
---  ----  ---------  ---------  ---------  -----------
  0  V      0.000283  -7.2e-05    0.004513  0.0100816
  1  V      0.002225   0.003893   0.204314  0.000500888
  2  V     -0.00116    5.1e-05    0.398816  0.134881
  3  V     -0.005048  -0.002627   0.590557  0.0119655
  4  Ti     0.001484   0.002929   0.800046  0.0372028
  5  V      0.000479   0.200946   0.00094   0.0859627
  6  V     -0.000925   0.199355   0.197856  0.183203
  7  V     -0.00058    0.198986   0.399472  0.119319
  8  V     -0.00109    0.197458   0.599131  0.00575411
  9  W     -0.003148   0.20629    0.801018  0.103227
 10  V     -0.000969   0.401421   0.001623  0.104991
 11  V      0.006908   0.401641   0.198342  0.0332206
 12  W     -0.000496   0.399211   

In [70]:
# define the adaptor 
adaptor = AseAtomsAdaptor()

# define root directory
root_dir = 'defect_test'
comp_dir = f'Cr{num_cr}Ti{num_ti}W{num_w}Zr{num_zr}Ta{num_ta}V{num_v}'
# possible list of directions 
directions = [np.array([1, 0, 0]), np.array([0, 1, 0]), np.array([0, 0, 1]), np.array([1, 1, 0]), np.array([1, 0, 1]), np.array([0, 1, 1]), np.array([1, 1, 1])]
# create the vacancies in each cell
i = 0
for i,super_cell in enumerate(relaxed_structures):
    # grab the structure
    structure = adaptor.get_atoms(super_cell['structure'])

    #select a random atom
    index = random.randint(0,len(structure)-1)
    print("Selecting atom at index: ", index)
    
    # select a random direction
    direction = random.choice(directions)
    print("Selecting direction: ", direction)

    # create the vacancies
    vacancy1, vacancy2 = create_vacancies_in_direction(structure, index, direction)

    # make a directory for each supercell
    #dir_name = "structure_" + str(i) + "_" + str(direction)
    struc_name = f"Structure_Num_{i}"
    vac_name = f"Vac_site_num_{index}" 
    direction_name = f"{direction[0]}{direction[1]}{direction[2]}"
    dir_name = os.path.join(comp_dir,struc_name,vac_name,direction_name)
    
    # make the directory
    os.makedirs(os.path.join(root_dir, dir_name), exist_ok=True)
    
    # write the supercell and the vacancies to the directory
    vacancy1.write(os.path.join(root_dir, dir_name, 'START'),format='vasp')
    vacancy2.write(os.path.join(root_dir, dir_name, 'END'),format='vasp')
    structure.write(os.path.join(root_dir, dir_name, 'POSCAR'),format='vasp')
    
    # make the energies.json file
    # first key will be "000", second key is "direction", the value will be the energies of vacancy1 and vacancy2

    relaxer = StructOptimizer(chgnet_vac_jan_26_pot)
    relax1 = relaxer.relax(atoms = vacancy1, fmax = 0.01, relax_cell=False,verbose=False)
    relax2 = relaxer.relax(atoms = vacancy2, fmax = 0.01, relax_cell=False,verbose=False)
    
    # get the energies of vacancy1 and vacancy2
    energy1 = relax1['trajectory'].energies[-1]
    energy2 = relax2['trajectory'].energies[-1]
    
    # write the energies to the json file
    energies = {"000" : energy1, direction_name : energy2}
    with open(os.path.join(root_dir, dir_name, 'energies.json'), 'w') as f:
        json.dump(energies, f)
    
    i+=1



Selecting atom at index:  37
Selecting direction:  [1 0 0]
CHGNet will run on cpu
Selecting atom at index:  98
Selecting direction:  [0 1 0]
CHGNet will run on cpu
Selecting atom at index:  31
Selecting direction:  [1 0 1]
CHGNet will run on cpu
Selecting atom at index:  94
Selecting direction:  [1 1 1]
CHGNet will run on cpu
Selecting atom at index:  122
Selecting direction:  [1 0 1]
CHGNet will run on cpu
Selecting atom at index:  114
Selecting direction:  [1 0 0]
CHGNet will run on cpu
Selecting atom at index:  76
Selecting direction:  [1 1 1]
CHGNet will run on cpu
Selecting atom at index:  110
Selecting direction:  [1 1 1]
CHGNet will run on cpu
Selecting atom at index:  106
Selecting direction:  [1 1 0]
CHGNet will run on cpu
Selecting atom at index:  36
Selecting direction:  [1 0 1]
CHGNet will run on cpu


In [72]:
import os
from ase.io import read, write

# Specify the root directory
root_dir = '/Users/myless/Dropbox (MIT)/Research/2023/Fall_2023/VASP/Structure_maker/defect_test'

# Walk through the directory tree
for dirpath, dirnames, filenames in os.walk(root_dir):
    # Check each file in the current directory
    for filename in filenames:
        # If the file is named 'START'
        if filename == 'START':
            # Construct the full file path
            start_file = os.path.join(dirpath, filename)
            # Read the file using ASE
            atoms = read(start_file, format='vasp')
            # Write the structure to a .cif file
            write(os.path.join(dirpath, 'START.cif'), atoms)